In [ ]:
from mip import Model, xsum, minimize, CBC, OptimizationStatus
from itertools import product
import matplotlib.pyplot as plt
import networkx as nx
import random

num_fabricas = 3
num_lojas = 7
num_centro_dist = 2

fabricas = range(num_fabricas)
lojas = range(num_lojas)
centro_dist = range(num_centro_dist)

ofertas_fabricas = [29, 30, 32]
capacidades_centro_dist = [50, 53]
demandas_lojas = [13, 17, 10, 8, 13, 15, 12]

custos_centro_fabrica = [
    [5, 7],
    [3, 5],
    [4, 6]
]
custos_centro_loja = [
    [4, 2, 3, 9, 5, 7, 3],
    [9, 7, 2, 4, 5, 4, 6]
]

# variaveis de decisao
arcos_ct = [(i, k) for (i, k) in product(fabricas, centro_dist)]
arcos_tm = [(k, j) for (k, j) in product(centro_dist, lojas)]

# declaração do modelo
modelo = Model('Problema de Transbordo', solver_name=CBC)

# x_ik igual a quantidade transportada i k pelos arcos com restrição > 0
x_ct = {(i, k): modelo.add_var(lb=0) for (i, k) in arcos_ct}
# x_kj igual a quantidade transportada k j pelos arcos com restrição > 0
x_tm = {(k, j): modelo.add_var(lb=0) for (k, j) in arcos_tm}

# função objetivo
modelo.objective = minimize(
    xsum(custos_centro_fabrica[i][k] * x_ct[i, k] for (i, k) in arcos_ct) +
    xsum(custos_centro_loja[k][j] * x_tm[k, j] for (k, j) in arcos_tm)
)

# restrição para os fabricas
for i in fabricas:
    modelo += xsum(x_ct[ii, k] for (ii, k) in arcos_ct if i == ii) <= ofertas_fabricas[i]

# restrições para os lojas
for j in lojas:
    modelo += xsum(x_tm[k, jj] for (k, jj) in arcos_tm if j == jj) >= demandas_lojas[j]

# restrições de equilíbrio nos pontos de transbordo
for k in centro_dist:
    modelo += xsum(x_ct[i, kk] for (i, kk) in arcos_ct if k == kk) <= capacidades_centro_dist[k]

for k in centro_dist:
  modelo += xsum(x_ct[i, kk] for (i, kk) in arcos_ct if k == kk) == xsum(x_tm[kk, j] for (kk, j) in arcos_tm if k == kk)

# otimização do modelo
status = modelo.optimize()
print(status)
lista_resultado_tuplas = []

# impressão do resultado
if status == OptimizationStatus.OPTIMAL:
    print(f"Custo total: {modelo.objective_value}")

    # imprimir as quantidades de origem para destino
    print("origem : destino : quantidade")
    for i in fabricas:
        for k in centro_dist:
            if x_ct[i, k].x > 0:
                print(f"{i} (Fabrica) : {k} (CD) : {int(x_ct[i, k].x)}")
                lista_resultado_tuplas.append(('F' + str(i), 'CD' + str(k), {'weight': int(x_ct[i, k].x)}))
    for k in centro_dist:
        for j in lojas:
            if x_tm[k, j].x > 0:
                print(f"{k} (CD) : {j} (Loja) : {int(x_tm[k, j].x)}")
                lista_resultado_tuplas.append(('CD' + str(k), 'L' + str(j), {'weight': int(x_tm[k, j].x)}))

G = nx.Graph()

lista_fabricas = ['F' + str(i) for i in range(num_fabricas)]
lista_centro_dist = ['CD' + str(i) for i in range(num_centro_dist)]
lista_lojas = ['L' + str(i) for i in range(num_lojas)]

G.add_nodes_from(lista_fabricas + lista_centro_dist + lista_lojas)
G.add_edges_from(lista_resultado_tuplas)

pos = {}

# Definir posições para os fabricas
for i, centro in enumerate(lista_fabricas):
    pos[centro] = (-1, i)

# Definir posições para os centro_dist
for i, transbordo in enumerate(lista_centro_dist):
    pos[transbordo] = (0, i)

# Definir posições para os lojas
for i, mercado in enumerate(lista_lojas):
    pos[mercado] = (1, i)

nx.draw_networkx_edges(G, pos=pos, arrows=True, arrowsize=20, arrowstyle='->', edge_color='blue', width=1)
edge_labels = {(i, j): p['weight'] for i, j, p in lista_resultado_tuplas}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='black', font_size=8, label_pos=0.85)
nx.draw_networkx_labels(G, pos, font_size=12, font_color='black')

plt.show()